In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras_tuner import HyperModel, Objective
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
# Veri yükleme
feature_list = ['HH_0.01_covariance_0_1',
  'HH_0.01_mean_0',
  'HH_0.01_radius_0_1',
  'HH_0.01_std_0',
  'HH_0.1_covariance_0_1',
  'HH_0.1_magnitude_0_1',
  'HH_0.1_std_0',
  'HH_0.1_weight_0',
  'HH_1_magnitude_0_1',
  'HH_1_pcc_0_1',
  'HH_1_radius_0_1',
  'HH_1_std_0',
  'HH_1_weight_0',
  'HH_3_covariance_0_1',
  'HH_3_magnitude_0_1',
  'HH_3_radius_0_1',
  'HH_3_std_0',
  'HH_3_weight_0',
  'HH_5_mean_0',
  'HH_5_pcc_0_1',
  'HH_5_std_0',
  'HH_5_weight_0',
  'HH_jit_0.01_std',
  'HH_jit_0.1_mean',
  'HH_jit_0.1_std',
  'HH_jit_1_mean',
  'HH_jit_1_std',
  'HH_jit_1_weight',
  'HH_jit_3_std',
  'HH_jit_5_std',
  'HpHp_0.01_magnitude_0_1',
  'HpHp_0.01_radius_0_1',
  'HpHp_0.01_weight_0',
  'HpHp_0.1_covariance_0_1',
  'HpHp_0.1_magnitude_0_1',
  'HpHp_0.1_mean_0',
  'HpHp_0.1_pcc_0_1',
  'HpHp_0.1_std_0',
  'HpHp_0.1_weight_0',
  'HpHp_1_covariance_0_1',
  'HpHp_1_magnitude_0_1',
  'HpHp_1_mean_0',
  'HpHp_1_pcc_0_1',
  'HpHp_1_radius_0_1',
  'HpHp_1_std_0',
  'HpHp_3_pcc_0_1',
  'HpHp_3_radius_0_1',
  'HpHp_5_pcc_0_1',
  'HpHp_5_radius_0_1',
  'HpHp_5_std_0',
  'HpHp_5_weight_0',
  'MI_dir_0.01_mean',
  'MI_dir_0.01_std',
  'MI_dir_0.01_weight',
  'MI_dir_0.1_mean',
  'MI_dir_0.1_std',
  'MI_dir_0.1_weight',
  'MI_dir_1_std',
  'MI_dir_1_weight',
  'MI_dir_3_mean',
  'MI_dir_5_mean',
  'MI_dir_5_std',
  'MI_dir_5_weight',
  'Label']
file_list={"HPO":['./small/AD-S1.csv','./small/AD-S2.csv']}

j="HPO"


# CNN

In [4]:
df=pd.read_csv(file_list[j][0],usecols=feature_list) 
X_train = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_train=df['Label'].cat.codes  

df=pd.read_csv(file_list[j][1],usecols=feature_list) 
X_test = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_test=df['Label'].cat.codes  



min_max_scaler = preprocessing.MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.fit_transform(X_test)
# Veriyi CNN modeline uygun hale getirme
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# CNN modeli oluşturma
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Conv1D(filters=hp.Int('filters', min_value=32, max_value=128, step=32),
                             kernel_size=hp.Int('kernel_size', min_value=3, max_value=5),
                             activation='relu',
                             input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(units=hp.Int(f'dense_{i}_units', min_value=32, max_value=128, step=32),
                               activation='relu'))
        model.add(layers.Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(layers.Dense(21, activation='softmax'))  # Çok sınıflı bir problem için 3 sınıf var
    
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])  # F1Score'u kullanarak ölçüm yapalım
    
    return model



# Hiperparametre araması yapma
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',#Objective("val_f1_score", direction="max"),
    max_trials=10,
    directory='CNNmy_dir',
    project_name='cnn_hyperparameter_tuning')

tuner.search(X_train, y_train,
             epochs=10,
             validation_data=(X_test, y_test))

# En iyi modeli seçme ve değerlendirme
best_model = tuner.get_best_models(num_models=1)[0]

y_pred = best_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
f1 = f1_score(y_test, y_pred_classes, average='macro')  # Weighted F1 score'u kullanıyoruz
print("Weighted F1 Score:", f1)

best_model.summary()
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print("En iyi modelin hiperparametreleri:")


Trial 10 Complete [00h 01m 08s]
val_accuracy: 0.222617506980896

Best val_accuracy So Far: 0.3421194553375244
Total elapsed time: 00h 12m 32s


C:\Users\kahra\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


580/580 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
Weighted F1 Score: 0.2921978403185517


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 59, 32)              │             192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 29, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 928)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │          29,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 21)                  │             693 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 30,613 (119.58 KB)

 Trainable params: 30,613 (119.58 KB)

 Non-trainable params: 0 (0.00 B)

En iyi modelin hiperparametreleri:


# LSTM

In [2]:
df=pd.read_csv(file_list[j][0],usecols=feature_list) 
X_train = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_train=df['Label'].cat.codes  


df=pd.read_csv(file_list[j][1],usecols=feature_list) 
X_test = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_test=df['Label'].cat.codes  


X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.fit_transform(X_test)

# Veriyi LSTM modeline uygun hale getirme
X_train = np.reshape(X_train, (X_train.shape[0],  X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0],  X_test.shape[1],1))

In [3]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.LSTM(units=hp.Int('units', min_value=32, max_value=128, step=32),
                          return_sequences=True,
                          input_shape=(X_train.shape[1], X_train.shape[2])))
    
    model.add(layers.Dropout(rate=hp.Float('dropout_1', min_value=0.0, max_value=0.5, step=0.1)))
    
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.LSTM(units=hp.Int(f'lstm_{i}_units', min_value=32, max_value=128, step=32),
                              return_sequences=True))
        model.add(layers.Dropout(rate=hp.Float(f'dropout_{i+2}', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(layers.LSTM(units=hp.Int('units_last', min_value=32, max_value=128, step=32)))
    model.add(layers.Dropout(rate=hp.Float('dropout_last', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(layers.Dense(21, activation='softmax'))  # Çok sınıflı bir problem için 3 sınıf var
    
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='sparse_categorical_crossentropy',
                  metrics=["accuracy"])  # F1Score'u kullanarak ölçüm yapalım
    
    return model


tuner = RandomSearch(
    build_model,
    objective=Objective("val_accuracy", direction="max"),
    max_trials=10,
    directory='LSTMmy_dir',
    project_name='lstm_hyperparameter_tuning')

tuner.search(X_train, y_train,
             epochs=10,
             validation_data=(X_test, y_test))

best_model = tuner.get_best_models(num_models=1)[0]

# En iyi modelin değerlendirilmesi
y_pred = best_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
f1 = f1_score(y_test, y_pred_classes, average='macro')  # Weighted F1 score'u kullanıyoruz
print("Weighted F1 Score:", f1)

best_model.summary()
print("En iyi modelin hiperparametreleri:")
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hyperparameters.values)

Trial 10 Complete [00h 23m 38s]
val_accuracy: 0.05390254408121109

Best val_accuracy So Far: 0.20601552724838257
Total elapsed time: 06h 10m 54s


C:\Users\kahra\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 24 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


580/580 ━━━━━━━━━━━━━━━━━━━━ 29s 49ms/step
Weighted F1 Score: 0.14448914211348543


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 63, 32)              │           4,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 63, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 63, 128)             │          82,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 63, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 96)                  │          86,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 96)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 21)                  │           2,037 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 175,221 (684.46 KB)

 Trainable params: 175,221 (684.46 KB)

 Non-trainable params: 0 (0.00 B)

En iyi modelin hiperparametreleri:
{'units': 32, 'dropout_1': 0.1, 'num_layers': 1, 'lstm_0_units': 128, 'dropout_2': 0.4, 'units_last': 96, 'dropout_last': 0.30000000000000004, 'learning_rate': 0.0001, 'lstm_1_units': 128, 'dropout_3': 0.4, 'lstm_2_units': 64, 'dropout_4': 0.0}


# MLP

In [4]:
df=pd.read_csv(file_list[j][0],usecols=feature_list) 
X_train = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_train=df['Label'].cat.codes  

df=pd.read_csv(file_list[j][1],usecols=feature_list) 
X_test = df.iloc[:,0:-1]
df['Label'] = df['Label'].astype('category')
y_test=df['Label'].cat.codes  

# Veriyi ANN modeline uygun hale getirme

X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.fit_transform(X_test)


In [5]:
# ANN modeli oluşturma
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units_input', min_value=32, max_value=128, step=32),
                           activation='relu',
                           input_shape=(X_train.shape[1],)))
    
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=128, step=32),
                               activation='relu'))
        model.add(layers.Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(layers.Dense(21, activation='softmax'))  # Çok sınıflı bir problem için 3 sınıf var
    
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='sparse_categorical_crossentropy',
                  metrics=["accuracy"])  # F1Score'u kullanarak ölçüm yapalım
    
    return model



# Hiperparametre araması yapma
tuner = RandomSearch(
    build_model,
    objective=Objective("val_accuracy", direction="max"),
    max_trials=10,
    directory='ANNmy_dir',
    project_name='ann_hyperparameter_tuning')

tuner.search(X_train, y_train,
             epochs=10,
             validation_data=(X_test, y_test))

# En iyi modeli seçme ve değerlendirme
best_model = tuner.get_best_models(num_models=1)[0]

y_pred = best_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
f1 = f1_score(y_test, y_pred_classes, average='macro')  # Weighted F1 score'u kullanıyoruz
print("Weighted F1 Score:", f1)

best_model.summary()
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print("En iyi modelin hiperparametreleri:")
print(best_hyperparameters.values)

Trial 10 Complete [00h 00m 47s]
val_accuracy: 0.1735122948884964

Best val_accuracy So Far: 0.2513475716114044
Total elapsed time: 00h 06m 59s
 26/580 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step  

C:\Users\kahra\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


580/580 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Weighted F1 Score: 0.18302709010399065


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │           2,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 21)                  │             693 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,797 (14.83 KB)

 Trainable params: 3,797 (14.83 KB)

 Non-trainable params: 0 (0.00 B)

En iyi modelin hiperparametreleri:
{'units_input': 32, 'num_layers': 1, 'units_0': 32, 'dropout_0': 0.0, 'learning_rate': 0.001, 'units_1': 32, 'dropout_1': 0.2, 'units_2': 128, 'dropout_2': 0.4}
